# Budget Analysis for SR methods

The analysis has the following parts: 

1. Maximum accuracy obtained per noise level
1. Basic learning curve 
1. Cost of annotation of the learning curve 

## Steps

1. Compute maximum accuracy for dataset
1. Compute maximum accuracy for fist-1
1. Compute levels of performance
1. Compute budget locations



In [2]:
DATA = 'C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/aclImdb/raw-data'
import sys
import os
sys.path.append(os.path.abspath('C:/cygwin/home/mramire8/python_code/structured/'))
sys.path.append(os.path.abspath('/Users/maru/MyCode/structured/'))

import learner

from utilities.datautils import load_dataset
import utilities.experimentutils as exputil
import numpy as np
import experiment.base as exp
import nltk


# EXample of IMDB 
kvalues = [10, 25, 50, 75, 100]
kv = [ 25, 50, 75, 100]
cost = np.array([5.7, 8.2, 10.9, 15.9, 16.7])

def fn_int(x, cost, kvalues):
    
    binx = min(np.digitize([x], kvalues)[0], len(cost)-1)
    lbbinx = max(binx-1, 0) 
#     print binx, lbbinx
    y1 = cost[lbbinx] if lbbinx>=0  else 0
    y2 = cost[binx]
    x1 = kvalues[lbbinx] if lbbinx >=0 else 0
    x2 = kvalues[binx]
#     print x1,x2,y1,y2
    
    m = (y2-y1) / (x2-x1)
    b = y2 - m * x2
    
    if x < kvalues[0]:
        y = cost[0]
    elif x > kvalues[-1] :
        y = cost[-1]
    else:
        y = (m * x) + b
    return y

print fn_int(98, cost, kvalues)

16.636


In [3]:
## Get the data ready
imdb_path = 'C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/aclImdb/raw-data'
imdb_path = '/Users/maru/MyCode/data/imdb'

vct = exputil.get_vectorizer({'vectorizer':'bow', 'limit':None, 'min_size':2})

imdb =  load_dataset("imdb",imdb_path, keep_subject=True)

# sraa = load_dataset("aviation", 100, categories[0], vct2, 100, raw=True,  percent=.5, keep_subject=True)
imdb.train.bow = vct.fit_transform(imdb.train.data)
imdb.test.bow = vct.transform(imdb.test.data)




In [6]:
# imdb.test.bow = vct.transform(imdb.test.data)

expert_config = {'type'           : "noisyreluctantscale", 
    'model'          : "lrl1",
    'parameter'      : 0.3,
    'costmodel'      : 1,
    'sent_tokenizer' : "first1snippet",
    'snip_size'      : (1,1),
    'threshold'      : .4,
    'scale': 0.
}
imdb_expert = exputil.get_expert(expert_config, size=len(imdb.train.data))
imdb_expert.fit(imdb.train.data, y=imdb.train.target, vct=vct)


In [7]:
print imdb.train.bow.shape

(24991, 267154)


In [ ]:
# for every document, get the snippet, convert to bow, 
# get the label from expert
# train a student classifier on resulting dataset
# test levels 

In [ ]:
# create dataset
# test student 

In [45]:
# Create the data for the labels 
def add_instance(mat, to_add):
    test_sent = mat
    if len(to_add) == 0: #.shape[0] == 0:
        return mat
    
    for instance in to_add:
        if isinstance(test_sent, list):
            test_sent = instance
        else:
            test_sent = vstack([test_sent, instance], format='csr')
    return test_sent

def convert2mat(txt_lst, target):
    tx = []
    ty = []
    for x,y in zip(txt_lst, target):
        tx.extend(x)
        ty.extend([y]*len(x))
    return np.array(tx), np.array(ty)

from sklearn.datasets import base as bunch 

def create_data(org_data, expert,  snip_tk, vct):
    snips = snip_tk.tokenize_sents(org_data.data)
    tx, y = convert2mat(snips, org_data.target)
    vecx = vct.transform(tx)
    query = bunch.Bunch(index=range(len(tx)), snippet=tx, bow=vecx)
#     query = {'bow':vecx,'snippet':tx, 'index':range(len(tx))}
    pred = expert.label(query, y=y)
    
    non_neu = np.array([i for i,x in enumerate(pred) if x is not None])
    return tx[non_neu], vecx[non_neu], np.array(pred[non_neu], dtype=int)




In [53]:
from sklearn import cross_validation
from sklearn import metrics 

def test_level(org_data, expert, vct, s, cost, kvalues, errors):
    ''' Test maximum performance of an student when using data from first-1 and 0 noise.
    '''
    rnd_state = np.random.RandomState(123)
#     train_sizes=np.linspace(.1, 1.0, 5)
    tk = exputil.get_tokenizer('first1snippet',snip_size=(s,s))

#     cv = cross_validation.ShuffleSplit(n, n_iter=5, test_size=0.0,
#                                        random_state=rnd_state)
    results = []

    for err in errors:
        print "=" * 15
        print "Error scale: %s" % err
        expert.set_scale_factor(err)
        tx, bow, y = create_data(org_data.train, expert, tk, vct)

        sizes = np.array([len(x.split()) for x in tx])
        cost2 = np.array([fn_int(x, cost, kvalues) for x in sizes])
        print "Average size: %s" % np.mean(sizes)
        print "Average cost: %s" % np.mean(cost2)
        clf = exputil.get_classifier('lrl1', parameter=1.)
        clf.fit(bow, y)
        pred = clf.predict(org_data.test.bow)
        accu = metrics.accuracy_score(org_data.test.target, pred)
        print "Accuracy w/o neutrals: %s" % accu
errors = [0.,0.142,0.284,0.425,0.567,0.709] # scale for IMDB
test_level(imdb, imdb_expert, vct, 1, cost, kvalues, errors)

Error scale: 0.0
Average size: 22.6506931696
Average cost: 7.56268067777
Accuracy w/o neutrals: 0.837208371684
Error scale: 0.142
Average size: 22.6506931696
Average cost: 7.56268067777
Accuracy w/o neutrals: 0.830965624875
Error scale: 0.284
Average size: 22.6506931696
Average cost: 7.56268067777
Accuracy w/o neutrals: 0.819280483413
Error scale: 0.425
Average size: 22.6506931696
Average cost: 7.56268067777
Accuracy w/o neutrals: 0.806754972188
Error scale: 0.567
Average size: 22.6506931696
Average cost: 7.56268067777
Accuracy w/o neutrals: 0.798351274561
Error scale: 0.709
Average size: 22.6506931696
Average cost: 7.56268067777
Accuracy w/o neutrals: 0.787986714154


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:32: RuntimeWarning: invalid value encountered in double_scalars


## Analysis for SRAA


In [55]:
## Get the data ready
sraa_path = 'C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/aclImdb/raw-data'
sraa_path = '/Users/maru/MyCode/data/sraa'

vct2 = exputil.get_vectorizer({'vectorizer':'bow', 'limit':None, 'min_size':2})

sraa =  load_dataset("sraa",sraa_path, keep_subject=True)

sraa.train.bow = vct2.fit_transform(sraa.train.data)
sraa.test.bow = vct2.transform(sraa.test.data)

# imdb.test.bow = vct.transform(imdb.test.data)

expert_config = {'type'           : "noisyreluctantscale", 
    'model'          : "lrl1",
    'parameter'      : 0.01,
    'costmodel'      : 1,
    'sent_tokenizer' : "first1snippet",
    'snip_size'      : (1,1),
    'threshold'      : .3,
    'scale': 0.
}
sraa_expert = exputil.get_expert(expert_config, size=len(sraa.train.data))
sraa_expert.fit(sraa.train.data, y=sraa.train.target, vct=vct2)

errors2 = [0.,0.138,0.276,0.414,0.552,0.690] # scale for IMDB
cost2=np.array([5.2, 6.5, 7.6,9.1,10.3])
test_level(sraa, sraa_expert, vct2, 1, cost2, kvalues, errors2)

Error scale: 0.0
Average size: 22.8930652028
Average cost: 6.14693315408
Accuracy w/o neutrals: 0.926454115783
Error scale: 0.138
Average size: 22.8930652028
Average cost: 6.14693315408
Accuracy w/o neutrals: 0.911154823375
Error scale: 0.276
Average size: 22.8930652028
Average cost: 6.14693315408
Accuracy w/o neutrals: 0.907548561593
Error scale: 0.414
Average size: 22.8930652028
Average cost: 6.14693315408
Accuracy w/o neutrals: 0.907739802748
Error scale: 0.552
Average size: 22.8930652028
Average cost: 6.14693315408
Accuracy w/o neutrals: 0.90088244133
Error scale: 0.69
Average size: 22.8930652028
Average cost: 6.14693315408
Accuracy w/o neutrals: 0.894598803377


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:32: RuntimeWarning: invalid value encountered in double_scalars
